# visfunctions

This notebook contains all of the functions that are used in 03 notebooks and above:
- 03_LineGraphs.ipynb
- 04_Maps.ipynb
- 05_AnimatedMaps.ipynb

See notebooks for more information.

## Import packages

In [ ]:
# The iconic trio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Date and time (datetime, perhaps?)
import time
from datetime import datetime as dt
from datetime import timedelta as td

# Mapping
import geopandas as gpd
import matplotlib.animation as animation

## Import Data

In [ ]:
df_all = pd.read_pickle('Data/CA2010+_dailyAQI.pkl')

In [ ]:
# Shapefile
map_df = gpd.read_file('Data/CA_Counties_TIGER2016.shp')

In [ ]:
counties = np.sort(df_all.COUNTY.unique())
sites = np.sort(df_all['Site Name'].unique())
nametypes_dict = {'Site Name':sites, 'COUNTY':counties}

In [ ]:
# Show sites and counties
def show_all():
    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        offset = len(sites) - len(counties)
        d = {'Site Name':sites, '':['']*len(sites), 'COUNTY':np.append(counties,['']*offset)}
        display(pd.DataFrame(d))

## Functions

### Site name/county searching

In [ ]:
def find_names(key, nametype, starts_with=True, one_only=False):
    '''
    Returns the names of site or COUNTY that begins with `start`.
    
    Support for CBSA may be implemented later.
    '''
    temp = nametypes_dict[nametype] # list of sites or counties
    output = np.array([])
    
    for name in temp: # Find all valid names
        if starts_with: # Only check start of names
            if name.upper().startswith(key.upper()):
                output = np.append(output, name)
        else: # Check whole string
            if key.upper() in name.upper():
                output = np.append(output, name)
    
    if len(output) == 0:
        return ''
    elif len(output) == 1 or one_only:
        return output[0] # If only one name, return string (not array)
    else:
        return output

In [ ]:
def get_d_r(date_range):
    # Get datetime values if not in datetime
    if isinstance(date_range[0], int): # year (int) -> datetime
        date_range[0] = dt(date_range[0],1,1) 
    elif isinstance(date_range[0], str): # date (str) -> datetime
        date_range[0] = dt.strptime(date_range[0], '%m/%d/%y') 
    if isinstance(date_range[1], int): # year (int) -> datetime
        date_range[1] = dt(date_range[1],12,31) 
    elif isinstance(date_range[1], str): # date (str) -> datetime
        date_range[1] = dt.strptime(date_range[1], '%m/%d/%y')
    
    return date_range

In [ ]:
def by_name(name, nametype, date_range=None):
    '''
    Returns date array, master df (all columns),
    AQI array, and location info of as a list.
    This function will return a list, with its elements
    being the value of that variable at each site.
    
    location = [Site Name, Site ID, CBSA_CODE, CBSA_NAME, COUNTY_CODE, COUNTY, 
                SITE_LATITUDE, SITE_LONGITUDE]
    '''
    d_l, d_r_l, m_l, AQI_l, loc_l = [],[],[],[],[]
    
    sitelist = df_all[df_all[nametype] == name]['Site Name'].unique()
    for site in sitelist:
        # Master
        master = df_all[df_all['Site Name'] == site]
        
        if date_range is not None: # date range is specified
            date_range = get_d_r(date_range)
            master = master[(master['Date'] >= date_range[0]) & (master['Date'] <= date_range[1])]
        
        # Skip if no data for site within this date range
        if len(master) == 0: continue
        else: m_l.append(master)
        
        # Date, Date Range, AQI
        d_l.append(master.Date.values)
        d_r_l.append([master.Date.min(), master.Date.max()])
        AQI_l.append(np.array(master.DAILY_AQI_VALUE.values))
        
        #Location
        loc_names = ['Site ID', 'Site Name', 'CBSA_CODE', 'CBSA_NAME', 'COUNTY_CODE', 'COUNTY', 
                     'SITE_LATITUDE', 'SITE_LONGITUDE']
        
        loc_l.append([master[col_name].iloc[0] for col_name in loc_names])
    
    
    return d_l, d_r_l, m_l, AQI_l, loc_l

### Plotting: 03_LineGraphs

In [ ]:
def find_outliers(a, smooth):
    '''
    Returns the indexes of the points that are significantly above or below
    (>2 SD) the value of the smoothed function.
    '''
    std = 1.5*np.std(a)
    smooth = smooth.values[:,0]
    outliers = []
    
    for i in range(len(a)):
        if np.isnan(smooth[i]):
            continue
        
        if abs(a[i] - smooth[i]) > std:
            outliers.append(i)
    
    return np.array(outliers)

In [ ]:
def plot_line(date, AQI, loc, window):
    # creating a smoother line using rolling mean
    smooth = pd.DataFrame(AQI).rolling(window, min_periods=1).mean()
    outliers_date = [date[i] for i in find_outliers(AQI, smooth)]
    outliers = [AQI[i] for i in find_outliers(AQI, smooth)]
    
    if isinstance(loc, str): label = loc + ' County'
    else: label = '{} ({})'.format(loc[1], loc[5])
    
    p = plt.plot(date, smooth, lw=1, label=label)
    plt.plot(outliers_date, outliers, '.', color=p[0].get_color(), alpha=0.4)

In [ ]:
def format_graph(d_r_tot, name_str):
    ax = plt.gca()
    
    # Figure out title and legend
    title = 'AQI: ' + d_r_tot[0].strftime("%m/%d/%Y") + ' - ' +\
            d_r_tot[1].strftime("%m/%d/%Y") + '\n'
    
    current_title = ax.title.get_text() # current title of graph
    if current_title != '':
        prev_names = current_title.split('\n')[1] # Get sites/counties already listed
        title += prev_names + ' | ' # keep previous names!
        
    plt.title(title + name_str[:-3] + '\n(lines = smoothed, dots = outliers)')
    plt.legend()
    
    # Axis labels and grid
    ax.set_xlabel('Date')
    ax.set_ylabel('AQI')
    ax.grid(True)

In [ ]:
def plot_lines(names, date_range=None, nametype='Site Name', window=10):
    '''
    Plots AQI of sites. If `nametype` is `'COUNTY'`, then the function will
    plot each site within that county individually.
    
    `date_range` bounds must be either: a datetime object; a year (integer); or
    a date (string) in the form 'month/day/year'. Bounds don't have to be of the
    same type.
    '''
    name_str = ''
    if isinstance(names, str): names = [names]
    d_r_tot = [] # maximum range of data in graph
    
    # for every name
    for name in names:
        orig_name = name
        name = find_names(name, nametype, one_only=True)
        
        if name == '': # No sites/counties of this name; go to next site
            print(orig_name, 'cannot be found.')
            continue
        
        date, d_r, master, AQI, loc = by_name(name, nametype, date_range)
        
        # for every site (one if 'Site Name', 1+ if 'COUNTY')
        for i in range(len(date)): 
            # Updating name_str
            if name not in name_str:
                name_str += name
                if nametype == 'COUNTY': name_str += ' County'
                name_str += ' | '
            
            # Determining max date range, for title
            if len(d_r_tot) == 0:
                d_r_tot = d_r[i]
            else:
                if d_r[i][0] < d_r_tot[0]: d_r_tot[0] = d_r[i][0]
                if d_r[i][1] > d_r_tot[1]: d_r_tot[1] = d_r[i][1]
                
            # Plot line
            plot_line(date[i], AQI[i], loc[i], window)
    
    
    if name_str == '': # No sites found
        return
    
    format_graph(d_r_tot, name_str)

In [ ]:
def get_mean(county, date_range=None, verbose=1):
    if verbose == 1:
        print('Calculating', county, end='    ')
    
    orig_county = county
    county = find_names(county, 'COUNTY', one_only=True)
    if county == '': print(orig_county, 'cannot be found.'); return
    
    # Get df of county averages per day
    df_county = df_all[df_all['COUNTY'] == county]
    if date_range is not None: # date range is specified
        date_range = get_d_r(date_range)
        df_county = df_county[(df_county['Date'] >= date_range[0]) &
                              (df_county['Date'] <= date_range[1])]

    df_mean = pd.DataFrame(columns=['Date', 'Daily Mean PM2.5 Concentration',
                                    'DAILY_AQI_VALUE', 'COUNTY'])
    # Averaging each day
    for i, date in enumerate(df_county.Date.unique()):
        temp = df_county[df_county['Date'] == date]
        PM_mean = np.mean(temp['Daily Mean PM2.5 Concentration'].values)
        AQI_mean = np.mean(temp['DAILY_AQI_VALUE'].values)
        df_mean.loc[i] = [date, PM_mean, AQI_mean, county]
    
    return df_mean.sort_values('Date').reset_index(drop=True)

In [ ]:
def plot_mean(counties, date_range=None, window=10):
    '''
    Plot each county as one line. This is done by averaging all 
    AQI measurements from each site in that county on each day.
    '''
    name_str = ''
    d_r_tot = [] # maximum range of data in graph
    if isinstance(counties, str): counties = [counties]
    
    for county in counties:
        df_mean = get_mean(county, date_range)
        if df_mean is None: continue
        
        date, PM, AQI, loc = df_mean.T.values # unpack columns
        d_r = [date.min(), date.max()]
        
        # Updating name_str
        if county not in name_str:
            name_str += county + ' County | '
        
        # Determining max date range, for title
        if len(d_r_tot) == 0:
            d_r_tot = d_r
        else:
            if d_r[0] < d_r_tot[0]: d_r_tot[0] = d_r[0]
            if d_r[1] > d_r_tot[1]: d_r_tot[1] = d_r[1]
        
        # Plot line
        plot_line(date, AQI, county, window)
    
    
    if name_str == '': # No sites found
        return
    
    format_graph(d_r_tot, name_str)

### Choropleth Maps: 04_Maps

In [ ]:
variable = 'DAILY_AQI_VALUE'

In [ ]:
def shp_merge(date, verbose=2):   
    if verbose == 1:
        print('.', end='')
    elif verbose == 2:
        print('Calculating', date.strftime('%m/%d/%Y'), end='    ')
    
    df_mean = pd.DataFrame(columns=['Date', 'Daily Mean PM2.5 Concentration',
                                    'DAILY_AQI_VALUE', 'COUNTY'])
    
    df_date = df_all[df_all['Date'] == date] # all data on this day
    
    for i, county in enumerate(df_date.COUNTY.unique()):
        row = get_mean(county, [date,date], verbose=0)
        df_mean.loc[i] = row.values[0]
    
    # Merge with shapefile
    merged = map_df.set_index('NAME').join(df_mean.set_index('COUNTY'))
    merged = merged[pd.notnull(merged[variable])] # remove NaN AQIs
    
    return merged

In [ ]:
def get_merged_list(dates, verbose=2):
    new_dates = []
    merged_list = []
    vrange = None
    
    for i, date in enumerate(dates):
        date = get_d_r([date,date])[0]
        new_dates.append(date)
        
        merged = shp_merge(date, verbose=verbose)
        merged_list.append(merged)
        
        # Get greatest min and max
        vmin, vmax = merged[variable].min(), merged[variable].max()
        if vrange is None:
            vrange = [vmin, vmax]
        else:
            if vrange[0] > vmin: vrange[0] = vmin
            if vrange[1] < vmax: vrange[1] = vmax
    
    return new_dates, merged_list, vrange

In [ ]:
def generate_map(dates, merged_list, vrange, fig, axes, cbar = True):
    '''
    Helper function. Does not work on its own!
    '''
    dl = len(dates)
    vmin, vmax = vrange
    cmap = 'YlOrRd' # https://matplotlib.org/examples/color/colormaps_reference.html
    
    for i, date in enumerate(dates):
        merged = merged_list[i]
        
        # Overlay gray plot with counties that have AQI data
        if dl == 1: c_ax = axes
        elif dl <= 2: c_ax = axes[i%2]
        else: c_ax = axes[i//2, i%2]
        
        # Plot
        map_df.plot(color='#d3d3d3', linewidth=0.8, ax=c_ax, edgecolor='0.8')
        p = merged.plot(column=variable, cmap=cmap, vmin=vmin, vmax=vmax,
                    linewidth=0.8, ax=c_ax, edgecolor='0.8')
        
        # Aesthetics
        c_ax.axis('off')
        c_ax.set_title('AQI on ' + str(date.strftime('%b %d, %Y')),
                     fontdict={'fontsize':'16', 'fontweight':'3'})
    
    # Colorbar
    if cbar:
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
        sm._A = []
        if dl > 1: cbar = fig.colorbar(sm, ax=axes.ravel().tolist(), shrink=.9)
        else: cbar = fig.colorbar(sm)
        cbar.ax.tick_params(labelsize=((dl-1)//2)*2 + 12) # 1 row = 12, 2 rows = 14...
    
    # odd number of dates, need to hide last plot
    if dl > 2 and dl%2 == 1: axes[-1,-1].axis('off')

In [ ]:
def map_counties(dates, save=False):
    '''
    Shows AQI data in a choropleth map, by county.
    Significantly inspired by Benjamin Cooley's article:
    https://towardsdatascience.com/lets-make-a-map-using-geopandas-pandas-and-matplotlib-to-make-a-chloropleth-map-dddc31c1983d
    '''
    if not isinstance(dates, list): dates = [dates]
    dl = len(dates)
    
    # Create figure
    if dl == 1: fig, axes = plt.subplots(1, figsize=(6,6))
    else: fig, axes = plt.subplots((dl+1)//2, 2, figsize=(17,4*dl))
    
    # Get all merged df's
    dates, merged_list, vrange = get_merged_list(dates)
    
    # Map
    print('\nGraphing...')
    generate_map(dates, merged_list, vrange, fig, axes)
    
    # Save if indicated
    if save:
        print('Saving...')
        fig.savefig('AQI_choropleth.png', dpi=300)

### Animation of Maps: 05_AnimatedMaps

In [ ]:
def ani_map_counties(dates, is_range=False, interval=500):
    '''
    Animated choropleth maps! if `is_range = False`, then `dates` is
    a list of dates, which will be shown in the animation. if `is_range
    = True`, then `dates` is a list of length 2 that define the range of
    dates.
    
    Seems to randomly save a 0 byte png to the folder...sorry for that!
    '''
    
    # If we're given a date range:
    if is_range and isinstance(dates, list) and len(dates) == 2:
        d1, d2 = np.sort(get_d_r(dates))
        delta = d2 - d1
        dates = [d1+td(i) for i in range(delta.days+1)] # all dates between d1/d2
    
    if not isinstance(dates, list): dates = [dates]
    
    # Create figure
    fig, ax = plt.subplots(1, figsize=(6,6))
    
    # Get all merged df's
    print('Calculating', end='')
    dates, merged_list, vrange = get_merged_list(dates, verbose=1)
    vmin, vmax = vrange
    
    # Color bar
    cmap = 'YlOrRd'
    sm = plt.cm.ScalarMappable(cmap=cmap, norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    fig.colorbar(sm)
    
    # Animation
    def animate(i, dates, merged_list, vrange, ax):
        ax.clear()
        generate_map([dates[i]], [merged_list[i]], vrange, fig, ax, cbar=False)
        if i != 0: print('.',end='')
    
    print('\nGenerating .', end='')
    ani = animation.FuncAnimation(fig, animate, fargs = (dates, merged_list, vrange, ax),
                                  frames=len(dates), interval=interval) # the meat of it!
    plt.close()
    return ani

<br><br><br><br><br><br><br>
<sup>(I'm an empty markdown cell.)</sup>
<br><br><br>
<sup><sup>weeeeeee</sup></sup>
<br><br><br><br><br>

## Deprecated

You're entering the land of unused functions! The function graveyard, you could say... ~spooky~

In [ ]:
# Plot all sites (takes a few minutes)
'''
plt.figure(figsize=(20,300))
# 165 total sites
for i, site in enumerate(sites):
    plt.subplot(56,3,i+1)
    plot_sites(site)'''
print(end='')

In [ ]:
def by_site(site, date_range=None):
    '''
    Returns date array, master df (all columns),
    AQI array, and location info of the site (Site Name column) as a list
    
    location = [Site Name, Site ID, CBSA_CODE, CBSA_NAME, COUNTY_CODE, COUNTY, 
                SITE_LATITUDE, SITE_LONGITUDE]
    '''
    master = df_all[df_all['Site Name'] == site]
    if date_range is not None: # date range is specified
        if isinstance(date_range[0], int):
            date_range[0] = dt(date_range[0],1,1) # year (str) -> datetime
        if isinstance(date_range[1], int):
            date_range[1] = dt(date_range[1],12,31) # year (str) -> datetime
        
        master = master[(master['Date'] > date_range[0]) & (master['Date'] < date_range[1])]
    
    date = master.Date.values
    date_range = [master.Date.min(), master.Date.max()]
    AQI = np.array(master.DAILY_AQI_VALUE.values)
    
    loc_names = ['Site ID', 'Site Name', 'CBSA_CODE', 'CBSA_NAME', 'COUNTY_CODE', 'COUNTY', 
                 'SITE_LATITUDE', 'SITE_LONGITUDE']
    location = [master[col_name].iloc[0] for col_name in loc_names]
    
    return date, date_range, master, AQI, location

In [ ]:
def by_county(county, date_range=None):
    '''
    Returns date array, master df (all columns),
    AQI array, and location info of the county (COUNTY column) as a list.
    This function will return a list, with its elements
    being the value of that variable at each site.
    
    location = [Site Name, Site ID, CBSA_CODE, CBSA_NAME, COUNTY_CODE, COUNTY, 
                SITE_LATITUDE, SITE_LONGITUDE]
    '''
    d_l, d_r_l, m_l, AQI_l, loc_l = [],[],[],[],[]
    
    sitelist = df_all[df_all['COUNTY'] == county]['Site Name'].unique()
    for site in sitelist:
        # Master
        master = df_all[df_all['Site Name'] == site]
        if date_range is not None: # date range is specified
            if isinstance(date_range[0], int):
                date_range[0] = dt.datetime(date_range[0],1,1) # year (str) -> datetime
            if isinstance(date_range[1], int):
                date_range[1] = dt.datetime(date_range[1],12,31) # year (str) -> datetime

            master = master[(master['Date'] > date_range[0]) & (master['Date'] < date_range[1])]
        
        m_l.append(master)
        
        # Date, Date Range, AQI
        d_l.append(master.Date.values)
        d_r_l.append([master.Date.min(), master.Date.max()])
        AQI_l.append(np.array(master.DAILY_AQI_VALUE.values))
        
        #Location
        loc_names = ['Site ID', 'Site Name', 'CBSA_CODE', 'CBSA_NAME', 'COUNTY_CODE', 'COUNTY', 
                     'SITE_LATITUDE', 'SITE_LONGITUDE']
        loc_l.append([master[col_name].iloc[0] for col_name in loc_names])
    
    return d_l, d_r_l, m_l, AQI_l, loc_l

In [ ]:
def iqr_outliers(a):
    '''
    Returns the indexes of the outliers in `a` (using interquartile range rules).
    '''
    q1, q3 = np.percentile(a, [25, 75])
    lower, upper = q1*1.5, q3*1.5
    
    outliers = np.where(np.logical_or(a>=upper, a<=lower))
    
    return outliers

In [ ]:
def plot_sites(sites, date_range=None, window=10):
    '''
    Plots AQI of sites. If `is_county` is `True`, then the function will
    plot each site within that county individually.
    '''
    names = ''
    if isinstance(sites, str): sites = [sites]
    d_r_tot = [] # maximum range of data in graph
    
    for site in sites:
        name = find_names(site, 'Site Name', one_only=True)
        
        if name == '': # No sites of this name; go to next site
            print(site, 'cannot be found.')
            continue
        
        names += name + ' | '
        date, d_r, master, AQI, loc = by_site(name, date_range)
        
        # Determining max date range, for title
        if len(d_r_tot) == 0:
            d_r_tot = d_r
        else:
            if d_r[0] < d_r_tot[0]: d_r_tot[0] = d_r[0]
            if d_r[1] > d_r_tot[1]: d_r_tot[1] = d_r[1]
        
        # creating a smoother line using rolling mean
        smooth = pd.DataFrame(AQI).rolling(window, min_periods=1).mean()
        outliers_date = [date[i] for i in find_outliers(AQI, smooth)]
        outliers = [AQI[i] for i in find_outliers(AQI, smooth)]
        
        p = plt.plot(date, smooth, lw=1, label='{} ({})'.format(loc[1], loc[5]))
        plt.plot(outliers_date, outliers, '.', color=p[0].get_color(), alpha=0.4)
    
    
    if names == '':
        return
    
    title_head = 'AQI: ' + d_r_tot[0].strftime("%m/%d/%Y") + '-' + d_r_tot[1].strftime("%m/%d/%Y")
    plt.title(title_head + '\n' + names[:-2] + '\n(lines = smoothed, dots = outliers)')
    plt.legend()
    
    ax = plt.gca()
    ax.grid()
    plt.show()

In [ ]:
def plot_counties(counties, date_range=None, window=10):
    '''
    Plots each county as their own line (by averaging AQI at each site).
    '''
    namelist = []
    if isinstance(counties, str): counties = [counties]
    
    for county in counties:
        name = find_names(county, 'COUNTY', one_only=True)
        if name == '': # No counties of this name; go to next county
            print(county, 'cannot be found.')
        else:
            sites = df_all[df_all['COUNTY'] == name]['Site Name'].unique()
            namelist.extend(sites)
    
    plot_sites(namelist, date_range=date_range, window=window)
    #plt.title()